In [1]:
import os

# Task 1: Binary Representations

For this task, I have to create certain functions and demonstrate their use with examples. These functions are:

1. The function rotl(x, n=1) that rotates the bits in a 32-bit unsigned integer to the left n places.

In [2]:
def rotateleft(x, n=1):
    # Perform bitwise rotation to the left by n positions
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

# Example test case:
x = 0b10110011001011101011001100101101
rotated = rotateleft(x, 4)
print(f"rotateleft(0b{x:032b}, 4) = 0b{rotated:032b}")

rotateleft(0b10110011001011101011001100101101, 4) = 0b00110010111010110011001011011011


2. The function rotr(x, n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.

In [3]:
def rotateright(x, n=1):
    # Perform bitwise rotation to the right by n positions
    return (x >> n) | ((x << (32 - n)) & 0xFFFFFFFF)

# Example test case:
x = 0b10110011001011101011001100101101
rotated = rotateright(x, 4)
print(f"rotateright(0b{x:032b}, 4) = 0b{rotated:032b}")

rotateright(0b10110011001011101011001100101101, 4) = 0b11011011001100101110101100110010


3. The function ch(x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

In [4]:
def choose(x, y, z):
    # Perform the 'choose' operation based on bits of x
    return (x & y) ^ (~x & z)

# Example test case:
x = 0b10101010101010101010101010101010
y = 0b11110000111100001111000011110000
z = 0b00001111000011110000111100001111
chosen = choose(x, y, z)
print(f"choose(0b{x:032b}, 0b{y:032b}, 0b{z:032b}) = 0b{chosen:032b}")

choose(0b10101010101010101010101010101010, 0b11110000111100001111000011110000, 0b00001111000011110000111100001111) = 0b10100101101001011010010110100101


4. The function maj(x, y, z) which takes a majority vote of the bits in x, y, and z.
The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

In [5]:
def majority(x, y, z):
    # Majority vote between x, y, and z for each bit position
    return (x & y) | (x & z) | (y & z)

# Example test case:
x = 0b11001100110011001100110011001100
y = 0b10101010101010101010101010101010
z = 0b01111000011110000111100001111000
majority = majority(x, y, z)
print(f"majority(0b{x:032b}, 0b{y:032b}, 0b{z:032b}) = 0b{majority:032b}")

majority(0b11001100110011001100110011001100, 0b10101010101010101010101010101010, 0b01111000011110000111100001111000) = 0b11101000111010001110100011101000


# Task 2: Hash Functions

For this task, I was given the following hash function from The C Programming Language by Brian Kernighan and Dennis Ritchie.

```
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}
```

My task is to convert it to Python, test it, and suggest why the values 31 and 101 are used.

In [6]:
def hash(s):
    hashvalue = 0
    for char in s:
        hashvalue = ord(char) + 31 * hashvalue
    return hashvalue % 101

# Example test case:
test = ["hello", "dog", "cat", "test", "example", ""]

for s in test:
    print(f"Hash value for '{s}' = {hash(s)}")

Hash value for 'hello' = 17
Hash value for 'dog' = 58
Hash value for 'cat' = 90
Hash value for 'test' = 86
Hash value for 'example' = 28
Hash value for '' = 0


Why the values 31 and 101 are used:

31:
The value 31 is commonly used in hash functions because it is a prime number, which helps reduce the likelihood of collisions in hash tables. It is a small prime number, which helps in spreading hash values uniformly.

101:
The value 101 is used as the modulus to ensure the hash value fits within a specific range (0 to 100).

# Task 3: SHA256

For this task, I have to write a Python function that calculates the SHA256 padding for a given file.<br>
The function should take a file path as input.<br>
It should print, in hex, the padding that would be applied to it.<br>
The [specification](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) states that the following should be appended to a message:

- a1 bit;
- enough 0 bits so the length in bits of padded message is the smallest possible multiple of 512;
- the length in bits of the original input as a big-endian 64-bit unsigned integer.

The example in the specification is a file containing the three bytes "abc":

01100001 01100010 01100011

The output would be:

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18

In [7]:
def sha256(file_path):
    with open(file_path, 'rb') as f:
        data = f.read()

    original_length = len(data)
    bit_length = original_length * 8

    # Start with the 0x80 byte (the '1' bit followed by seven 0s)
    padding = b'\x80'

    # Calculate how many zero bytes are needed to reach 56 mod 64
    total_length = original_length + 1
    
    padding_zeros = (56 - total_length % 64) % 64

    padding += b'\x00' * padding_zeros

    # Append the original message length as a 64-bit (8 bytes) big-endian integer
    padding += bit_length.to_bytes(8, byteorder='big')

    # Print the padding in hex format
    print(' '.join(f'{byte:02x}' for byte in padding))

sha256("task3.txt") # Replace with the path to your file


80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


This script reads the contents of a file and computes the SHA-256 padding that would be added before hashing the data. The padding is then printed in hex format.

# Task 4: Prime Numbers

For this task, I have to calculate the first 100 prime numbers using two different algorithms.<br>
Any algorithms that are well-established and works correctly are okay to use.<br>
I then have to explain how the algorithms work.

# Task 5: Roots

For this task, I have to calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

# Task 6: Proof of Work

For this task, I have to find the word(s) in the English language with the greatest number of 0 bits at the beginning of their SHA256 hash digest.<br>
I have to include proof that any word listed is in at least one English dictionary.

# Task 7: Turing Machines

For this task, I have to design a Turing Machine that adds 1 to a binary number on its tape.<br>
The machine should start at the left-most non-blank symbol.<br>
It should treat the right-most symbol as the least significant bit.

For example, suppose the following is on the tape at the start:

100111

Your Turing machine should leave the following on the tape when it completes:

101000

# Task 8: Computational Complexity

For this task, I have to implement bubble sort in Python, modifying it to count the number of comparisons made during sorting.<br>
I must use this function to sort all permutations of the list:

L = [1, 2, 3, 4, 5]

For each permutation, I have to print the permutation itself followed by the number of comparisons required to sort it.